In [1]:
import pandas as pd 
import nltk 
import pickle
import json
from bs4 import BeautifulSoup
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
string.punctuation += '=+-'
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn import preprocessing, model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, log_loss
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [2]:
indexs = string.punctuation.index('.')
string.punctuation = string.punctuation[0:indexs]+string.punctuation[indexs+1:]
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
def remove_punctuation(text):
    #print(text)
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc 
    

In [4]:
def remove_stopwords(text):
    #print(text)
    words = [c for c in text if c not in stopwords.words('english')]
    return words 

In [5]:

def countWords(s):
    words = s.split()
    if len([word for word in words if len(word)<=3])/ len(s.split()) >0.50:
        return  np.NaN
    else :
        return s

In [6]:
def train_model(classifier, 
                feature_vector_train, 
                label, 
                feature_vector_valid, 
                fname,is_neural_net=False,):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    print(confusion_matrix(predictions,valid_y))
    fname += '.pkl'
    pickle.dump(classifier, open(fname, 'wb'))
 
    return accuracy_score(predictions, valid_y)

In [7]:

def word_lemmetizer(text):
    lem_text =" ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

In [15]:
path1 = r"hospital14.csv"
path2 = r"hospital7.csv"
path3 = r"hospital8.csv"
train = True 

In [16]:
def preprocess(path1, path2 = None):
    if path2 is not  None :
        df1 = pd.read_csv(path1)
        df2 = pd.read_csv(path2)
        df = df1.append(df2, ignore_index = True)
    else :
        df = pd.read_csv(path1)
    df.drop(df[df['text'].isna()].index, inplace= True)
    df.drop(df[df['label'].isna()].index, inplace= True)
    print(len(df))
    print(df[df['text'].isna()].index)
    ### removing the common start  of the document 
    dfk= df['text'].str.split("\n", n = 5, expand = True) 
    df['ptext'] = dfk[5]
    df.drop(df[df['ptext'].isna()].index, inplace= True)
    del dfk
    ####
    ## check for the substring
    index = df.sample().index[0]
    if df.loc[index]['ptext'] in df.loc[index]['text']:
        print('substring fouund')
    else :
        print('index problem')
        exit()
    df['ptext'] = df['ptext'].apply(lambda x: remove_punctuation(x) )
    df['ptext'] = df['ptext'].apply(lambda x: tokenizer.tokenize(x.lower()) )
    df['ptext'] = df['ptext'].apply(lambda x: remove_stopwords(x))
    df['ptext'] = df['ptext'].apply(lambda x: word_lemmetizer(x))
    corpus = df['ptext'].tolist()
    df_train_nlp=pd.DataFrame({'image_name':df['image_name'],'text':corpus,'label':df['label']})
    df_train_nlp = df_train_nlp.replace(['Gaurantee_Letter','gaurantee-Letter','Guarantee Letter','Gaurantee_letter','Garauntee_Letter'],'Authorisation_Form')
    df_train_nlp = df_train_nlp.replace(['Lab-report','Lab Report'],'Lab_Report')
    df_train_nlp = df_train_nlp.replace(['Doctors Notes'],'Doctors_Notes')
    return df_train_nlp

In [17]:
df_train =  preprocess(path1, path2 )
df_val =  preprocess(path3 )


751
Int64Index([], dtype='int64')
substring fouund
412
Int64Index([], dtype='int64')
substring fouund


In [19]:

train_x, valid_x, train_y, valid_y = train_test_split(df_train['text'],
                                                  df_train['label'],
                                                  test_size=0.10
                                                   )


In [20]:

index_label = defaultdict(str)
if train == True :
    print('Train')
    encoder = preprocessing.LabelEncoder()
    train_y = encoder.fit_transform(train_y)
    valid_y = encoder.fit_transform(valid_y)
    label_name = encoder.inverse_transform(train_y)   
    for i,j in zip(train_y, label_name):
    #for i,j in zip(valid_y, label_name):
        if not index_label[str(i)]:
            index_label[str(i)] = j
if train == False :
 
    
    valid_y = encoder.fit_transform(df_train_nlp['label'])
    valid_x = df_train_nlp['text']
#     label_name = encoder.inverse_transform(valid_y)   
#     for i,j in zip(valid_y, label_name):
#         if not index_label[str(i)]:
#             index_label[str(i)] = j
    
index_label

Train


defaultdict(str,
            {'4': 'Lab_Report',
             '3': 'Invoice',
             '2': 'IC',
             '1': 'Doctor_Note',
             '0': 'Authorisation_Form'})

# Vectorisation

In [21]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', stop_words="english", token_pattern=r'\w{1,}')
count_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [22]:
tfidf_vect = TfidfVectorizer(analyzer='word',
                             stop_words="english",
                             token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(df_train['text'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

In [23]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', 
                                   stop_words="english",
                                   token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), 
                                   max_features=5000)
tfidf_vect_ngram.fit(df_train['text'])
xtrain_tfidf_ngram =tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)

In [24]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', 
                                         #stop_words="english",
                                         #token_pattern=r'\w{1,}', 
                                         ngram_range=(2,3), 
                                         max_features=5000)
tfidf_vect_ngram_chars.fit(df_train['text'])
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(valid_x) 

# Naive Bayes


In [25]:
accuracy = train_model(MultinomialNB(), 
                       xtrain_count, 
                       train_y, 
                       xvalid_count,'NBcount')
print ("NB Count Vectors: ", accuracy)

[[ 6  0  0  0  0]
 [ 0 18  2  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 31  1]
 [ 1  0  0  0 17]]
NB Count Vectors:  0.9473684210526315


In [26]:
accuracy = train_model(MultinomialNB(), 
                       xtrain_tfidf, 
                       train_y, 
                       xvalid_tfidf,'NBtfidf')
print ("NB accuracy with WordLevel TF-IDF: ", accuracy)

[[ 6  0  0  0  0]
 [ 0 18  1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  1 31  1]
 [ 1  0  0  0 17]]
NB accuracy with WordLevel TF-IDF:  0.9473684210526315


In [27]:
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(MultinomialNB(), 
                       xtrain_tfidf_ngram, 
                       train_y, 
                       xvalid_tfidf_ngram,'NBngram')
print ("NB accuracy with N-Gram Vectors: ", accuracy)

[[ 6  0  0  0  0]
 [ 0 18  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  1 31  1]
 [ 1  0  1  0 17]]
NB accuracy with N-Gram Vectors:  0.9473684210526315


In [28]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(MultinomialNB(), 
                       xtrain_tfidf_ngram_chars, 
                       train_y, 
                       xvalid_tfidf_ngram_chars,'NBChar')
print ("NB accuracy with CharLevel Vectors: ", accuracy)

[[ 6  0  0  0  0]
 [ 1 18  1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  1 31  1]
 [ 0  0  0  0 17]]
NB accuracy with CharLevel Vectors:  0.9473684210526315


In [30]:
classifiers = [
    KNeighborsClassifier(n_neighbors=5),
    MultinomialNB(),
    SVC(kernel="rbf", C=0.015, probability=True, class_weight='balanced'),
    RidgeClassifier(tol=1e-3),
    LogisticRegression(n_jobs=1, C=1e5, max_iter=1000, class_weight='balanced'),
    SGDClassifier(loss='squared_loss', penalty='elasticnet', 
                  alpha=1e-3, random_state=7, 
                  max_iter=200, tol=None,
                 class_weight='balanced'),
        BaggingClassifier(n_estimators=20),
    AdaBoostClassifier(n_estimators=50, learning_rate=0.05),
    GradientBoostingClassifier(learning_rate=0.05, 
                               n_estimators=100,
                               min_samples_split=6,
                               max_depth=10,),
    DecisionTreeClassifier(max_depth=20,
                          min_samples_split=3,
                          min_samples_leaf=1,
                          class_weight='balanced'),
    RandomForestClassifier(n_estimators=200,
                           class_weight='balanced'),
    XGBClassifier(learning_rate =0.05,
                         n_estimators=100,
                         class_weight='balanced')]


for classifier in classifiers:
    pipe = Pipeline(steps=[('vect', CountVectorizer(max_df=0.95, 
                                                    min_df=0.,
                                                    max_features=1200)),
                           ('tfidf', TfidfTransformer()),
                           ('classifier', classifier)])
    
    pipe.fit(train_x, train_y)   
    predtn = pipe.predict(valid_x)
    print(classifier)
    print("model score: %.4f" % pipe.score(valid_x, valid_y))
    print(confusion_matrix(predtn, valid_y))
    print("===================================")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
model score: 0.9737
[[ 6  0  0  0  0]
 [ 0 18  0  0  0]
 [ 0  0  2  0  0]
 [ 0  0  0 31  1]
 [ 1  0  0  0 17]]
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
model score: 0.9474
[[ 6  0  0  0  0]
 [ 0 18  2  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 31  1]
 [ 1  0  0  0 17]]
SVC(C=0.015, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
model score: 0.4079
[[ 6  0  0  0  0]
 [ 0 13  0  0  0]
 [ 1  5  2 21 18]
 [ 0  0  0 10  0]
 [ 0  0  0  0  0]]
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                so

In [31]:
valid_y = encoder.fit_transform(df_val['label'])
valid_x = df_val['text']

In [32]:
## final model ##
classifier =  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

model = Pipeline(steps=[('vect', CountVectorizer(max_df=0.95, 
                                                min_df=0.,
                                                max_features=1200)),
                       ('tfidf', TfidfTransformer()),
                       ('classifier', classifier)])
    
model.fit(train_x, train_y)   

# save the model to disk
filename = 'hosp_text.pkl'
pickle.dump(model, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(valid_x, valid_y)
pred = loaded_model.predict(valid_x)
print(confusion_matrix(pred,valid_y),"\n", confusion_matrix(valid_y, valid_y))
print(result)

[[ 88   0   0   0   0]
 [ 15 108   1   0   0]
 [  1   0  15   0   0]
 [  0   0   1  81   0]
 [  0   0   0   2 100]] 
 [[104   0   0   0   0]
 [  0 108   0   0   0]
 [  0   0  17   0   0]
 [  0   0   0  83   0]
 [  0   0   0   0 100]]
0.9514563106796117


In [ ]:
df8.to_csv(r'C:\task\779\csv\8\final8.csv', index = False )